last edited by Claire Valva on September 4, 2018
# Spectral analysis is performed on 40.5N data


In [1]:
#set whether or not to reload data 
#or use data saved from an old session
do_trends = False  
run_fresh = False

In [2]:
#import packages
import numpy as np
from scipy.signal import get_window, csd
from scipy.fftpack import fft, ifft, fftfreq, fftshift, ifftshift
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import matplotlib.cm as cm
from math import pi
import matplotlib.ticker as tck
import datetime
from sympy import solve, Poly, Eq, Function, exp, re, im
from PyEMD import EMD
from netCDF4 import Dataset, num2date # This is to read .nc files and time array
from scipy.optimize import fsolve

In [3]:
#import own functions from functions_forspectralanalysis.py
#functions are based off of those in spectral_analysis_tests.ipynb 
#but edited so that they can be used in multiple notebooks
from functions_forspectralanalysis import *

/home/clairev/python-climate-data-processing/submission_from_claire/functions_forspectralanalysis.py:75: RuntimeWarning: divide by zero encountered in true_divide
  zonal_spacing = 1/zonal_spacing


[NbConvertApp] Converting notebook functions_forspectralanalysis.ipynb to script
[NbConvertApp] Writing 4656 bytes to functions_forspectralanalysis.py


## Import and edit file

In [4]:
#import file
filepath = '/home/clairev/uncategorized-data/1979-2016-300hPa-40.5N-z.nc' # Location of the file
fileobj = Dataset(filepath, mode='r')

# Check what's in there
fileobj

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): longitude(240), time(55520)
    variables(dimensions): float32 z(time,longitude)
    groups: 

In [5]:
#set indicies/number of things
number_entries = int(fileobj.dimensions['time'].size)
number_days = int(number_entries / 4)
number_lon = fileobj.dimensions['longitude'].size
year_number = 2016 - 1979 + 1

In [6]:
#load coordinates
#so height[i] is the geopotential height at a given time
height = fileobj.variables['z'][:]
g_inv = 1/9.81
height = height*g_inv

In [7]:
#create time array
time_list = []
for i in range(0,55520):
    time_list.append(i*6)
tunit = "hours since 1979-01-01T00:00:00Z"
tarray = num2date(time_list,units = tunit,calendar = 'gregorian')

#create longitude array
lon_increment = 1.5 # The increment of each longitude grid is 1.5
lon_list = [i * lon_increment for i in range(240)]

In [8]:
if run_fresh:

#get lists and then merge together as a dataframe
    lon_list = [lon_list[k]
               for i in range(number_entries)
               for k in range(number_lon)]

    z_temp = [height[i][k]
              for i in range(number_entries)
              for k in range(number_lon)]

    date_list = [tarray[i]
               for i in range(number_entries)
               for k in range(number_lon)]
    
    #make this a dataframe
    d = {'datetime' : date_list, 'lon': lon_list,
                    'z' : z_temp}
    geopot_df = pd.DataFrame(d)
    
    #get day/month/year separately for groupby
    geopot_df['month'] = geopot_df['datetime'].apply(lambda x: x.month)
    geopot_df['day'] = geopot_df['datetime'].apply(lambda x: x.day)
    geopot_df['year'] = geopot_df['datetime'].apply(lambda x: x.year)
    
    #sort into seasons    
    geopot_df["season"] = geopot_df["month"].apply(lambda x: season_sort(x))
    
    
    # Create storage object with filename `processed_data`
    data_store = pd.HDFStore('processed_data.h5')

    # Put DataFrame into the object setting the key as 'preprocessed_df'
    data_store['preprocessed_geopot'] = geopot_df
    data_store.close()
    
else:
        # Access data store
    data_store = pd.HDFStore('processed_data.h5')

    # Retrieve data using key
    geopot_df = data_store['preprocessed_geopot']
    data_store.close()

## Perform zonal fft

In [9]:
#get fft results at each date over entire longitude
fft_zonal_result = [geopot_fft(height[k]) 
                    for k in range(number_entries)]

## Perform time fft

### detrending

detrend all data using results from zonal fft

In [10]:
#get list of years and seasons to perform transform/detrend on
years = range(1979,2017)
seasons = ["winter", "spring", "summer", "fall"]

#### define some functions which depend on the files imported earlier

In [11]:
def trendsfordf(year, season):
    frame = geopot_df.query(query_string(year, season, 9))
    
    trend_list = [pairwavenum(k, frame, tarray, fft_zonal_result) 
                  for k in range(len(zonal_spacing))]
    
    return(trend_list)

##### TO DO: this function gets a lot of runtime warnings, likely from the fsolve - would be nice to fix it up later on // for time being just come back and give an option to pickle it later

##### TO DO: should be right, but check on the time indexing after finish  transforming back

In [12]:
def coeff_subtract(time, frame, tarray, fft_zonal_result, trend_list):
    #since gettimes depends on frame, get frame first
    
    #get first and last timestamp
    tmin, tmax = gettimes(frame, tarray)
 
    #get the time steps at the beginning and end of the seasons
    iter_list = time - tmin
    
    #get the results and then subtract the list
    results = fft_zonal_result[time]
    tosub = [sublist[1][iter_list - 1] for sublist in trend_list]
    
    results = results - tosub
    
    return results

In [13]:
def trendsfordf(year, season):
    frame = geopot_df.query(query_string(year, season, 9))
    
    #get trends
    trend_list = [pairwavenum(k, frame, tarray, fft_zonal_result) 
                  for k in range(len(zonal_spacing))]
    
    tmin, tmax = gettimes(frame, tarray)
    
    #adjust the list
    adjusted_list = [coeff_subtract(time, frame, tarray, fft_zonal_result, trend_list) for time in range(tmin, tmax+tmin)]
    
    #perform ifft on the list
    adjusted_list_ifft = [ifft(sublist) for sublist in adjusted_list]
    
    return(season, year, trend_list, adjusted_list, adjusted_list_ifft)

In [14]:
#get the trends in fourier coefficients
if do_trends == True:
    trends_all_list = [trendsfordf(time, season) for season in seasons
                       for time in years]
    
    import pickle

    file_Name = "test_trends_pickle"
    file_pickle = open(file_Name,'wb') 

    pickle.dump(trends_all_list,file_pickle)
    file_pickle.close()

In [15]:
if do_trends == False:
    import pickle
    
    file_Name = "test_trends_pickle"
    
    file_pickle = open(file_Name, "rb")
    
    trends_all_list = pickle.load(file_pickle)

note that the structre of trends_all_list is:

trends = [time, season, trends, adjusted forier coeffs, ifft of adjusted forier coeffs] * 152

where time is a year
season is a season
trends for that time and season are 240 entries of changing trends

In [16]:
#pull everything out of the list
season_index = [sublist[0] for sublist in trends_all_list]
year_index = [sublist[1] for sublist in trends_all_list]
trend_index = [sublist[2] for sublist in trends_all_list]
adjcof_index = [sublist[3] for sublist in trends_all_list]
untrend_index = [sublist[4] for sublist in trends_all_list]

In [17]:
#flatten list of detrended z values
flatten_df_untrend = [item for sublist in untrend_index for item in sublist]
flatten_df_untrend = [item for sublist in flatten_df_untrend for item in sublist]

In [18]:
#get list of same length for df binding - season, year, lon lists
season_df_list = []
year_df_list = []
lon_df_list = []

for i in range(len(untrend_index)):
    for j in range(len(untrend_index[i])):
        for k in range(240):
            entry = season_index[i]
            yr = year_index[i]
            lon = lon_list[k]
            
            season_df_list.append(entry)
            year_df_list.append(yr)
            lon_df_list.append(lon)

In [19]:
d = {"season": season_df_list, "year": year_df_list,
     "lon": lon_df_list, "adj_z": flatten_df_untrend}

untrend_df = pd.DataFrame(d)

In [20]:
def time_fft(season, year, longitude):
    #function that applies fft to geo height data by year, season, longitude
    
    frame = untrend_df.query(query_string_v2(year, season, longitude))
    
    #get spacing
    num_entries = len(frame["adj_z"])
    spacing = fftfreq(num_entries, 0.25)
    
    #apply fft
    fft_coeff = fft(frame["adj_z"])
    ck_coeff = fft_coeff/ num_entries
    
    #return info and the fft
    return season, year, longitude, spacing, fft_coeff, ck_coeff

In [ ]:
#get spectra of each individual season
fft_total_list = [time_fft(sea, yr, lon) for sea in seasons 
                 for time in years 
                 for lon in lon_list]